In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from dataneeds.engine.dask_bag import DaskBagEngine
from dask.async import get_sync
import dask.bag as db

In [3]:
db.zip?

In [3]:
import dataneeds as need
from dataneeds.engine.dask_bag import DaskBagEngine

In [4]:
e = DaskBagEngine()

In [5]:
%run dataneeds/tests/graph.py

In [46]:
with need.request(Node()) as N:
    N.label, N.edges.id, N.edges.weight

In [47]:
js

OrderedDict([(dataneeds/tests/*.nlf >> Sep(',', 2) >> Cons >> Cons[2] >> Sep(',', -1) >> Each >> Each[None] >> Node.edges->Edge().id,
              ([Node.edges->Edge().weight],
               [dataneeds/tests/*.elf >> Sep(',', -1) >> Cons >> Cons[0] >> Edge().id:Integer,
                dataneeds/tests/*.elf >> Sep(',', -1) >> Cons >> Cons[1] >> Edge().weight:Floating]))])

In [55]:
type(rs[0].binds)

dataneeds.types.Attribute

In [48]:
(rs, js), *_ = N.resolve_combined().values()

In [49]:
e.resolve(N.items, rs, js).compute(get=get_sync)

[]

In [50]:
list(N.resolve_combined().keys())

[(Files(dataneeds/tests/*.nlf), Files(dataneeds/tests/*.elf)),
 (Files(dataneeds/tests/*.nlf), Files(dataneeds/tests/*.nef)),
 (Files(dataneeds/tests/*.nef), Files(dataneeds/tests/*.elf)),
 (Files(dataneeds/tests/*.nef), Files(dataneeds/tests/*.nef))]

In [16]:
with need.request(Edge()) as E:
    E.id, E.weight, E.source.id, E.target.id

In [18]:
a, (rs, js), *_ = E.resolve_combined().values()

In [19]:
e.resolve(E.items, rs, js).compute(get=get_sync)

[([0, 1], [0.3, 0.2], 0, [1, 2]),
 ([2, 3, 4], [0.2, 1.0, 0.4], 1, [0, 1, 2]),
 ([], [], 2, [])]

In [18]:
with need.request(Node()) as N:
    N.edges.id, N.edges.weight, N.id, N.edges.target.id

In [20]:
with need.request(Edge()) as E:
    E.id,
    E.source.id, E.source.label
    E.target.id, E.target.label
    E.weight
E.items

[Edge().id: Integer,
 Edge.source->Node().id,
 Edge.source->Node().label,
 Edge.target->Node().id,
 Edge.target->Node().label,
 Edge().weight: Floating]

In [21]:
list(E.resolve_combined().keys())

[(Files(dataneeds/tests/*.elf),
  Files(dataneeds/tests/*.nlf),
  Files(dataneeds/tests/*.nlf)),
 (Files(dataneeds/tests/*.elf),
  Files(dataneeds/tests/*.nlf),
  Files(dataneeds/tests/*.nef)),
 (Files(dataneeds/tests/*.elf),
  Files(dataneeds/tests/*.nef),
  Files(dataneeds/tests/*.nlf)),
 (Files(dataneeds/tests/*.elf),
  Files(dataneeds/tests/*.nef),
  Files(dataneeds/tests/*.nef)),
 (Files(dataneeds/tests/*.nef),
  Files(dataneeds/tests/*.nlf),
  Files(dataneeds/tests/*.nlf)),
 (Files(dataneeds/tests/*.nef),
  Files(dataneeds/tests/*.nlf),
  Files(dataneeds/tests/*.nef)),
 (Files(dataneeds/tests/*.nef),
  Files(dataneeds/tests/*.nef),
  Files(dataneeds/tests/*.nlf)),
 (Files(dataneeds/tests/*.nef),
  Files(dataneeds/tests/*.nef),
  Files(dataneeds/tests/*.nef))]

In [22]:
(rs, js), *_ = E.resolve_combined().values()

In [23]:
rs

[dataneeds/tests/*.elf >> Sep(',', -1) >> Cons >> Cons[0] >> Edge().id:Integer,
 dataneeds/tests/*.elf >> Sep(',', -1) >> Cons >> Cons[2] >> Edge.source->Node().id,
 dataneeds/tests/*.elf >> Sep(',', -1) >> Cons >> Cons[3] >> Edge.target->Node().id,
 dataneeds/tests/*.elf >> Sep(',', -1) >> Cons >> Cons[1] >> Edge().weight:Floating]

In [24]:
js

OrderedDict([(dataneeds/tests/*.elf >> Sep(',', -1) >> Cons >> Cons[2] >> Edge.source->Node().id,
              ([Edge.source->Node().label],
               [dataneeds/tests/*.nlf >> Sep(',', 2) >> Cons >> Cons[0] >> Node().id:Integer,
                dataneeds/tests/*.nlf >> Sep(',', 2) >> Cons >> Cons[1] >> Node().label:String])),
             (dataneeds/tests/*.elf >> Sep(',', -1) >> Cons >> Cons[3] >> Edge.target->Node().id,
              ([Edge.target->Node().label],
               [dataneeds/tests/*.nlf >> Sep(',', 2) >> Cons >> Cons[0] >> Node().id:Integer,
                dataneeds/tests/*.nlf >> Sep(',', 2) >> Cons >> Cons[1] >> Node().label:String]))])

In [25]:
e.resolve(E.items, rs, js).compute(get=get_sync)

[(0, 0, 'A', 1, 'B', 0.3),
 (1, 0, 'A', 2, 'C', 0.2),
 (2, 1, 'B', 0, 'A', 0.2),
 (3, 1, 'B', 1, 'B', 1.0),
 (4, 1, 'B', 2, 'C', 0.4)]